In [2]:
import pandas as pd
import sys
import numpy as np

## Orders


In [3]:
import src.order

In [4]:
order_type = "bid"

qty_b1 = 100
price_b1 = 99
bid1 = src.order.Order(order_type, qty_b1, price_b1)

qty_b2 = 10
price_b2 = 98
bid2 = src.order.Order(order_type, qty_b2, price_b2)

order_type = "ask"

qty_a1 = 100
price_a1 = 101
ask1 = src.order.Order(order_type, qty_a1, price_a1)

qty_a2 = 10
price_a2 = 102
ask2 = src.order.Order(order_type, qty_a2, price_a2)

## Order Book


In [5]:
from src.order_book import OrderBook

orderbook = OrderBook([bid1, bid2], [ask1, ask2])
print(orderbook)

┌─────────┬────────┬─────────┬─────────┐
│   Index │   Size │   Price │   Total │
├─────────┼────────┼─────────┼─────────┤
│       0 │     10 │     102 │    1020 │
│       1 │    100 │     101 │   10100 │
│       2 │    100 │      99 │    9900 │
│       3 │     10 │      98 │     980 │
└─────────┴────────┴─────────┴─────────┘


## Main Script


In [6]:
from src.kandel import kandel_simulator
from src.time_series import load_csv

In [7]:
ts = load_csv("data/binance/ETHUSDC-2024-01>08_1s.csv")

In [8]:
hv = ts.to_pandas()["price"].rolling(3600 * 24).std()/(1**0.5)
hv_1h = hv.resample("1h").last()

In [9]:
iv_1h = pd.read_csv(
    "data/deribit/ETH_volIndex.csv", index_col=0, parse_dates=True
)
iv_1h = iv_1h.drop(columns=["vol_index_o", "vol_index_h", "vol_index_l", "vol_index_c"])
iv_1h.index = pd.to_datetime(iv_1h.index, unit="h")
iv_1h["vol_index"] = iv_1h["vol_index"] * 100 
iv_1h = iv_1h.align(hv_1h, join="inner", axis=0)[0]


In [10]:
iv = iv_1h.loc[iv_1h.index.repeat(3600)][:len(hv)]
iv.index = hv.index

In [28]:
window = 3600 * 24
quote = 37500  # USDC
base = 37500  # also in USDC, will be swapped to base at day 0
vol_mult = 1.645
n_points = 10
step_size = 1
days = 11

transactions, res, order_book_history = kandel_simulator(
    ts=ts[-(days + 1) * 24 * 3600 :],
    quote=quote,
    base=base,
    vol_mult=vol_mult,
    n_points=n_points,
    step_size=step_size,
    window=window,
    historical_vol=hv[-(days + 1) * 24 * 3600 :],
    implied_vol=iv["vol_index"][-(days + 1) * 24 * 3600 :],
)
res.to_pandas().to_csv("results/simul_results.csv")

100%|██████████| 950400/950400 [00:16<00:00, 56427.10it/s] 


# Results


In [13]:
import pandas as pd  # type: ignore

pd.options.plotting.backend = "plotly"

from plotly.subplots import make_subplots
import plotly.graph_objects as go
from datetime import datetime

In [30]:
res = pd.read_csv("results/simul_results.csv", index_col=0, parse_dates=True)

In [31]:
res["mtm_eth"] = res["mtm"] / res["price"]
res["base_returns"] = res["mtm_eth"] / ((quote + base) / res["price"][window]) - 1
res["quote_returns"] = res["mtm"] / (quote + base) - 1

/var/folders/kn/wgl1sqns1bxgynxc599_5y9c0000gp/T/ipykernel_27736/4188196617.py:2: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [32]:
window_hours = window // 3600 - 1
res_1h = res.resample("1h").last()

# PLOT
fig = make_subplots(
    rows=3,
    cols=1,
    shared_xaxes=True,
    specs=[[{"secondary_y": True}], [{}], [{"secondary_y": True}]],
    vertical_spacing=0.05,
    subplot_titles=("MTM", "Returns", "Price"),
)
fig.add_trace(
    go.Scatter(
        x=res_1h.index[window_hours:],
        y=res_1h.mtm[window_hours:],
        mode="lines",
        name="MTM in USDC",
    ),
    row=1,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=res_1h.index[window_hours:],
        y=res_1h.mtm_eth[window_hours:],
        mode="lines",
        name="MTM in ETH",
    ),
    row=1,
    col=1,
    secondary_y=True,
)

fig.add_trace(
    go.Scatter(
        x=res_1h.index[window_hours:],
        y=res_1h.quote_returns[window_hours:],
        mode="lines",
        name="Over holding USDC",
    ),
    row=2,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=res_1h.index[window_hours:],
        y=res_1h.base_returns[window_hours:],
        mode="lines",
        name="Over holding ETH",
    ),
    row=2,
    col=1,
)

fig.add_trace(
    go.Scatter(
        x=res_1h.index[window_hours:],
        y=res_1h.price[window_hours:],
        mode="lines",
        name="Price",
    ),
    row=3,
    col=1,
)

fig.add_trace(
    go.Scatter(
        x=iv_1h.index[-len(res_1h) + window_hours :],
        y=iv_1h.vol_index[-len(res_1h) + window_hours :],
        mode="lines",
        name="IV"
    ),
    secondary_y=True,
    row=3,
    col=1,
)

fig.add_trace(
    go.Scatter(
        x=hv_1h.index[-len(res_1h)+ window_hours :],
        y=hv_1h.values[-len(res_1h) + window_hours:],
        mode="lines",
        name="HV"
    ),
    secondary_y=True,
    row=3,
    col=1,
)

fig.update_layout(
    height=1000,
    width=1400,
    title_text=f"MTM, Returns and Price.<br><sup>window = {window} | quote = {quote} | base = {base} | vol_mult = {vol_mult} | n_points = {n_points} | step_size = {step_size} | days = {days}</sup>",
)

fig.update_yaxes(row=2, col=1, tickformat=".2%")

fig.show()

fig.write_image(
    f"results/saves/{window}_{quote}_{base}_{vol_mult}_{n_points}_{step_size}_{days}.png",
    width=1400,
    height=1000,
)

In [33]:
order_book_history_df = pd.DataFrame.from_records(
    [ob.to_dict() for ob in order_book_history]
)
order_book_history_df.index = res[window -1:].index
order_book_history_1h = order_book_history_df.resample("1h").last()

In [34]:
order_book_history_1h

,bids,asks
time,,
2024-08-21 00:00:00+00:00,[],[]
2024-08-21 01:00:00+00:00,[],[]
2024-08-21 02:00:00+00:00,[],[]
2024-08-21 03:00:00+00:00,[],[]
2024-08-21 04:00:00+00:00,[],[]
...,...,...
2024-08-31 20:00:00+00:00,"[2488.592151, 2476.834843, 2465.133082, 2453.4...","[2512.274464, 2524.2, 2536.182146, 2548.22117,..."
2024-08-31 21:00:00+00:00,"[2512.274464, 2500.405269, 2488.592151, 2476.8...","[2536.182146, 2548.22117, 2560.317342, 2572.47..."
2024-08-31 22:00:00+00:00,"[2512.274464, 2500.405269, 2488.592151, 2476.8...","[2536.182146, 2548.22117, 2560.317342, 2572.47..."


In [89]:
fig = make_subplots(
    rows=1,
    cols=1,
    shared_xaxes=True,
    vertical_spacing=0.05,
    subplot_titles=(""),
    specs=[[{"secondary_y": True}]],
)

for i, ob in enumerate(order_book_history_1h.iterrows()):
    bids = ob[1].bids
    asks = ob[1].asks

    fig.add_trace(
        go.Scatter(
            x=[i for _ in range(len(bids))],
            y=tuple(bids),
            mode="markers",
            name="Bids",
            showlegend=False,
            marker=dict(
                color="green",
                symbol="line-ew-open",
                size=8,
                line=dict(width=5, color="black"),
            ),
        ),
    )

    fig.add_trace(
        go.Scatter(
            x=[i for _ in range(len(asks))],
            y=tuple(asks),
            mode="markers",
            name="Asks",
            showlegend=False,
            marker=dict(
                color="red",
                symbol="line-ew-open",
                size=8,
                line=dict(width=5, color="black"),
            ),
        ),
    )

fig.add_trace(
    go.Scatter(
        x=[i for i in range(len(order_book_history_1h))],
        y=res_1h.price[window_hours:],
        mode="lines",
        name="Price",
        marker=dict(color="blue"),
        hovertext=[
            f"Price: {res_1h.price[window_hours:][i]:.2f}<br>Time: {res_1h.index[window_hours:][i].strftime('%b %d %H:%M')}"
            for i in range(len(res_1h.price[window_hours:]))
        ]
    )
)

fig.update_layout(
    height=1000,
    width=1600,
    title_text=f"Order book<br><sup>window = {window} | quote = {quote} | base = {base} | vol_mult = {vol_mult} | n_points = {n_points} | step_size = {step_size} | days = {days}</sup>",
)

fig.update_xaxes(
    tickvals=np.arange(0, len(res_1h[window_hours:]), 24),
    ticktext=[res_1h[window_hours:].index[i].strftime("%b %d, %Hh") for i in np.arange(0, len(res_1h[window_hours:]), 24)],
)

fig.show()

/var/folders/kn/wgl1sqns1bxgynxc599_5y9c0000gp/T/ipykernel_27736/3308611468.py:54: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [67]:
order_book_history_1h.index[0]

Timestamp('2024-08-21 00:00:00+0000', tz='UTC')

In [36]:
generated_fees = []

for i, t in enumerate(transactions):
    if t:
        generated_fees.append(order_book_history[i+1].to_pandas()["Size"].sum() - order_book_history[i].to_pandas()["Size"].sum())
    else:
        generated_fees.append(0)


   

In [40]:
generated_fees = pd.Series(generated_fees, index=res.index[window:])
cumulative_fees = generated_fees.cumsum()
cumulative_fees_1h = cumulative_fees.resample("1h").last()
cumulative_fees_1h.plot()

/opt/anaconda3/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [41]:
cumulative_vol = res["volume"].cumsum()
cumulative_vol_1h = cumulative_vol.resample("1h").last()
cumulative_vol_1h.plot()

/opt/anaconda3/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



- Observer quand signal pas trigger mais pertes
- Benchmark du kill switch


- Vérifier la qualité des datas binance
- Tester kill switch plus simple: regarder vol hist plus courte et vol impli séparement 


Idées:
- regarder la corrélation entre return et prix final

In [11]:
ts_1h = ts.to_pandas().resample("1h").last()

In [18]:
fig = make_subplots(specs=[[{"secondary_y": True}]])


fig.add_trace(
    go.Scatter(
        x=ts_1h.index,
        y=ts_1h.price,
        mode="lines",
        name="Price",
    ),
    secondary_y=True,
)
fig.add_trace(
    go.Scatter(
        x=hv_1h.index,
        y=hv_1h,
        mode="lines",
        name="HV",
    ),
)

fig.add_trace(
    go.Scatter(
        x=iv_1h.index,
        y=iv_1h.vol_index,
        mode="lines",
        name="IV",
    ),
)

In [92]:
iv_1h

,vol_index
2024-01-01 00:00:00+00:00,70.150
2024-01-01 01:00:00+00:00,69.665
2024-01-01 02:00:00+00:00,69.225
2024-01-01 03:00:00+00:00,69.295
2024-01-01 04:00:00+00:00,69.355
...,...
2024-08-31 20:00:00+00:00,62.085
2024-08-31 21:00:00+00:00,61.975
2024-08-31 22:00:00+00:00,61.885
2024-08-31 23:00:00+00:00,61.935
